In [1]:
# armazenar os documentos totais em variavel

import os
rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
#rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'
docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [2]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/r2-d2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/r2-d2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def remove_header(text):
    # Find the first occurrence of '\n\n'
    pos = text.find('\n\n')

    # Remove everything after the first '\n\n'
    if pos >= 0:
        text = text[pos+2:]
    return text
    

In [25]:
def remove_email(text):
    pattern = r'\S+@\S+'
    # Remove email addresses from the string
    text = re.sub(pattern, '', text)
    return text


In [27]:
#teste = remove_header(docs[0]['conteudo'])
#teste = remove_email(teste)

In [82]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):


    text = remove_header(text)

    text = remove_email(text)


    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    #text = re.sub('<[^<]+?>', '', text)
    
    # manter so texto
    text = re.sub('[^A-Za-z ]+', '', text)

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    #words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)

    
    return text



In [83]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [ ]:
# Pré-cálculo do IDF para cada termo em todos os documentos
# LEGADO

# import math

# frases = [o['conteudo'] for o in preprocessed_documents]
# n = len(frases)
# idfs = {}
# for frase in frases:
#     for termo in set(frase.split()):
#         idfs[termo] = math.log(n / sum(1 for f in frases if termo in f))    

In [84]:
preprocessed_documents = preprocess_collection(docs)

In [85]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [86]:
frases[500]

'article dave fuller writes hello started reading group today think going large participant daily posting liked section faq constructing logical argument well done atheist try turn people atheist try figure people believe way dont much care different view come could wrong willing admit possibility something religious follower dont seem capability dowelcome aboard notice alot post bobby anybody ever respond post always fall back argumenti think answered question religion followed cause bad right event explained human done name religion mean actually followed religion always point ideal say wasnt followed cant reason event really way argue bother sure may get upset answer blind supported factually win every time little argument dont think posting direct response one hismost response posting spouted fact thatall atheist foolsevil seeing peachy islam isi would leave procon argument islam fred rice morelevel headed seems know subject anyway happy aboard know going welcome abord dave fuller 

In [87]:
# Pré-cálculo do IDF para cada termo em todos os documentos


from collections import Counter
import math

n = len(frases)
termos_doc_count = Counter()

# Contagem do número de documentos que contém cada termo
for frase in frases:
    termos = set(frase.split())
    for termo in termos:
        termos_doc_count[termo] += 1

idfs = {}
# Cálculo dos valores IDF para cada termo
for termo, doc_count in termos_doc_count.items():
    idf = math.log(n / doc_count)
    idfs[termo] = idf

In [88]:
# Função para calcular o TF-IDF

def tfidf(termo, documento, idfs):
    tf = documento.count(termo)
    idf_valor = idfs.get(termo, 0)
    return tf * idf_valor

In [ ]:
# Função para gerar o ranking de similaridade

# LEGADO

# def similaridade(texto, documentos, idfs):
   
#     # separando as palavras do texto
#     texto = texto.split()
    
#     # Lista de documentos com os valores TF-IDF
#     documentos_tfidf = []
    
#     for doc in documentos:
#         # Normalização do documento

#         documento = doc['conteudo'].split()
        
#         # Cálculo do valor TF-IDF para cada termo do texto
#         tfidf_documento = sum(tfidf(termo, documento, idfs) for termo in texto)
        
#         # Adição do documento e seu valor TF-IDF na lista de documentos
#         documentos_tfidf.append((doc['nome_arquivo'], doc['tema'], doc['conteudo'], tfidf_documento))

    
#     # Ordenação dos documentos pela similaridade
#     documentos_tfidf = sorted(documentos_tfidf, key=lambda x: x[3], reverse=True)
    
#     # Retorno dos 10 documentos mais similares com suas respectivas similaridades
#     return documentos_tfidf[:20]

In [89]:
# NOVA FUNCAO

def similaridade(texto, documentos, idfs):
    # Separar as palavras do texto
    texto_palavras = texto.split()

    # Calcular o IDF para cada palavra do texto
    idf_texto = {termo: idfs[termo] for termo in texto_palavras}

    # Pré-processar os documentos
    documentos_preprocessados = []
    for doc in documentos:
        # Dividir o documento em palavras
        documento_palavras = doc['conteudo'].split()

        # Remover as palavras que não aparecem no texto
        documento_palavras = [word for word in documento_palavras if word in idf_texto]

        # Armazenar o documento pré-processado
        documentos_preprocessados.append((doc['nome_arquivo'], doc['tema'],  doc['conteudo'], documento_palavras))

    # Calcular o TF-IDF para cada documento
    documentos_tfidf = []
    for nome_arquivo, tema, conteudo, documento_palavras in documentos_preprocessados:
        tfidf_documento = sum(idf_texto[termo] * documento_palavras.count(termo) for termo in texto_palavras)
        documentos_tfidf.append((nome_arquivo, tema, conteudo, tfidf_documento))

    # Ordenar os documentos pela similaridade
    documentos_tfidf = sorted(documentos_tfidf, key=lambda x: x[3], reverse=True)

    # Retornar os 20 documentos mais similares com suas respectivas similaridades
    #return documentos_tfidf[:100]
    return documentos_tfidf

In [90]:
print(preprocessed_documents[90]['conteudo'])
print(preprocessed_documents[90]['tema'])
print(preprocessed_documents[90]['nome_arquivo'])


test adam john cooperverily often laughed weakling thought good simply claw understand one another fear beyond comprehension gandalf
alt.atheism
53401


In [91]:
def calcula_map(ranking, tema):
    acertos = 0
    maps = []
    for i, doc in enumerate(ranking):
        if doc[1] == tema:
            acertos +=1
            #print(i)
            maps.append(acertos/(i+1))
        #print(len(ranking))
    #print(maps)
    #print(mean(maps))
    #print(len(maps))
    return mean(maps)

In [92]:
#ranking = similaridade(preprocessed_documents[0]['conteudo'], preprocessed_documents, idfs)
map = calcula_map(ranking, preprocessed_documents[0]['tema'])
#for doc in ranking:
    #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
    #print(f"Documento {doc[0]} - Target {doc[1]} --> Similaridade: {doc[3]}")


In [93]:
# Exemplo de uso - NOVO

#ranking = similaridade(preprocessed_documents[17502]['conteudo'], preprocessed_documents, idfs)

#Impressão do ranking
#for i, doc in enumerate(ranking):
acertos = 0
#for doc in ranking:
for i, doc in enumerate(ranking):
    #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
    if doc[1] == 'alt.atheism':
        acertos+=1
    if i+1 == 10:
        print(f"10 primeiros acertou {acertos}")
        print(str(acertos/float(10)))
    if i+1 == 20:
        print(f"20 primeiros acertou {acertos}")
        print(str(acertos/float(20)))
    if i+1 == 50:
        print(f"50 primeiros acertou {acertos}")                
        print(str(acertos/float(50)))
    if i+1 == 100:
        print(f"100 primeiros acertou {acertos}")        
        print(str(acertos/float(100)))


10 primeiros acertou 0
0.0
20 primeiros acertou 0
0.0
50 primeiros acertou 1
0.02
100 primeiros acertou 3
0.03


In [106]:
# armazenar os documentos totais em variavel

import os
rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [107]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

temas = {}
for d in dirs_abs:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [108]:
#processando

doc_consultas_preproc = preprocess_collection(docs_consultas)

In [109]:
doc_consultas_preproc[500]

{'tema': 'comp.windows.x',
 'nome_arquivo': '66964',
 'conteudo': 'xdm solaris broke since two week sun distributesa patched release work fine support etcshadow allwe running ever since experienced anyproblems call local sun repdick ahold room ankersmidplein zaandam netherlands eec tel fax'}

In [110]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(doc_consultas_preproc):
    clear_output(wait=True)
    print(c)
    ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    acertos = 0
    map_f.append(calcula_map(ranking, doc_consulta['tema']))
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        if doc[1] == doc_consulta['tema']:
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(doc_consultas_preproc) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            map = []

    tema_atual = doc_consulta['tema']
    # if c > 102:
    #      break    


1999


In [111]:
temas

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.22871287128712872,
  'p20': 0.18861386138613861,
  'p50': 0.1603960396039604,
  'p100': 0.15603960396039604,
  'map': 0.10091128236298608},
 'comp.graphics': {'p10': 0.599,
  'p20': 0.451,
  'p50': 0.2762,
  'p100': 0.19190000000000002,
  'map': 0.09263055014872107},
 'comp.os.ms-windows.misc': {'p10': 0.059000000000000004,
  'p20': 0.0555,
  'p50': 0.09960000000000001,
  'p100': 0.1261,
  'map': 0.09392315555774028},
 'comp.sys.ibm.pc.hardware': {'p10': 0.195,
  'p20': 0.1535,
  'p50': 0.1338,
  'p100': 0.1361,
  'map': 0.09232171491029216},
 'comp.sys.mac.hardware': {'p10': 0.223,
  'p20': 0.1585,
  'p50': 0.0998,
  'p100': 0.0886,
  'map': 0.08854769979365872},
 'comp.windows.x': {'p10': 0.496,
  'p20': 0.417,
  'p50': 0.3214,
  'p100': 0.2284,
  'map': 0.08917122818034229},
 'misc.forsale': {'p10': 0.084,
  'p20': 0.0805,
  'p50': 0.0586,
  'p100': 0.0611,
  'map': 0.0864431403336457}

In [112]:
import json
with open('data_lemma_map_noheader.json', 'w') as f:
    json.dump(temas, f)

In [113]:
import pandas as pd

df = pd.DataFrame(temas)

In [114]:
df

,mini_newsgroups,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,...,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
p10,0,0.228713,0.599000,0.059000,0.195000,0.223000,0.496000,0.084000,0.209000,0.15400,...,0.361000,0.376000,0.16600,0.306000,0.357000,0.257000,0.222000,0.601000,0.395000,0.174747
p20,0,0.188614,0.451000,0.055500,0.153500,0.158500,0.417000,0.080500,0.159000,0.11750,...,0.300500,0.327000,0.12400,0.247000,0.276500,0.245000,0.179000,0.539000,0.368500,0.144444
p50,0,0.160396,0.276200,0.099600,0.133800,0.099800,0.321400,0.058600,0.130200,0.09560,...,0.263600,0.267000,0.08340,0.208000,0.221000,0.233000,0.160200,0.462200,0.302200,0.126667
p100,0,0.156040,0.191900,0.126100,0.136100,0.088600,0.228400,0.061100,0.137200,0.09900,...,0.265100,0.255600,0.07250,0.203700,0.208100,0.227600,0.166300,0.436400,0.252500,0.130707
map,0,0.100911,0.092631,0.093923,0.092322,0.088548,0.089171,0.086443,0.087454,0.08655,...,0.093175,0.096961,0.09416,0.093417,0.093478,0.097298,0.098087,0.103702,0.103942,0.103238


In [115]:
df =df.transpose()

In [116]:
df.drop('mini_newsgroups')

,p10,p20,p50,p100,map
alt.atheism,0.228713,0.188614,0.160396,0.156040,0.100911
comp.graphics,0.599000,0.451000,0.276200,0.191900,0.092631
comp.os.ms-windows.misc,0.059000,0.055500,0.099600,0.126100,0.093923
comp.sys.ibm.pc.hardware,0.195000,0.153500,0.133800,0.136100,0.092322
comp.sys.mac.hardware,0.223000,0.158500,0.099800,0.088600,0.088548
comp.windows.x,0.496000,0.417000,0.321400,0.228400,0.089171
misc.forsale,0.084000,0.080500,0.058600,0.061100,0.086443
rec.autos,0.209000,0.159000,0.130200,0.137200,0.087454
rec.motorcycles,0.154000,0.117500,0.095600,0.099000,0.086550
rec.sport.baseball,0.161000,0.132500,0.128200,0.142400,0.087188


In [117]:
df.describe()

,p10,p20,p50,p100,map
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.267831,0.222122,0.182432,0.170731,0.089648
std,0.162136,0.138015,0.107455,0.092879,0.021286
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.166000,0.132500,0.099800,0.126100,0.088548
50%,0.223000,0.179000,0.160200,0.156040,0.093417
75%,0.361000,0.300500,0.263600,0.227600,0.097298
max,0.601000,0.539000,0.462200,0.436400,0.103942
